<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesora: Maria Paz Raveau</em><br>
    <em>Asignatura: Procesamiento de Lenguaje Natural</em><br>

</div>

# **Tarea 1**

*Fecha de Entrega: Domingo 20, Julio 2025.*

**Nombre Estudiante**: Victor Saldivia Vera

### **01. Intrucciones y Enunciado**

Suponga que ud. es contactado por alguna agencia gubernamental que quiere saber qué dijeron los participantes de los cabildos de 2016. Ud. debe entonces, basándose en los textos de fundamento, responder a este requerimiento.  Use para ello la base de datos que vimos en clases, de los cabildos provinciales. 

- Escoja alguna de los conceptos que vienen en la base de datos (evite las que no tengan muchas entradas). Hemos visto cómo generar WordClouds con tokens y con bigramas. *¿Es posible valerse de las etiquetas gramaticales para mejorar el resultado?* 
- Pruebe haciendo un WordCloud con tokens/sustantivos,  y otro con  bigramas/sustantivo-adjetivo. Recuerde pre-procesar correctamente.
- Con estos mismo bigramas/sustantivo-adjetivo, haga una red de bigramas.
- Ayúdese con frases de ejemplo, y de los WordClouds, para escribir una descripción del concepto.


### **02. Descripción de la Data**


In [5]:
import pandas as pd
df = pd.read_excel('data/resultadocabildoprovincial.xlsx')
print(f'Dimensión: {df.shape}'), 
print(f'Nombre de Columnas: {df.columns}')

Dimensión: (30414, 12)
Nombre de Columnas: Index(['region_nombre', 'provincia_nombre', 'comuna_nombre', 'cabildo_id',
       'cabildo_nombre', 'cabildo_direccion', 'num_mesa', 'pregunta_glosa',
       'concepto_nombre', 'concepto_otro', 'tipo-eleccion', 'fundamento'],
      dtype='object')


### **03. Librerías a Utilizar**


In [ ]:
import pandas as pd
import numpy as np
import re
import unicodedata
from collections import Counter

# Librerías de Procesamiento de Texto
import nltk
from nltk.corpus import stopwords
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
import stanza

# Visualización
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import networkx as nx

# La librería de warnings se utiliza para manejar las advertencias
import warnings
warnings.filterwarnings('ignore')

# Paquete muestra Barra de Progreso
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# Descargas (Nota: Ejecutar una sola vez en entorno local)
# nltk.download('punkt')
# nltk.download('stopwords')
# stanza.download('es')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Victo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Victo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-07-19 22:12:42 INFO: Downloaded file to C:\Users\Victo\stanza_resources\resources.json
2025-07-19 22:12:42 INFO: Downloading default packages for language: es (Spanish) ...
2025-07-19 22:12:44 INFO: File exists: C:\Users\Victo\stanza_resources\es\default.zip
2025-07-19 22:12:50 INFO: Finished downloading models and saved to C:\Users\Victo\stanza_resources


#### Configuración de Stanza para el PLN

In [ ]:
nlp = stanza.Pipeline('es', processors='tokenize,pos,lemma', use_gpu=False)  # Cambiar a True si hay GPU


2025-07-19 22:13:12 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-07-19 22:13:12 INFO: Downloaded file to C:\Users\Victo\stanza_resources\resources.json
2025-07-19 22:13:12 WARNING: Language es package default expects mwt, which has been added
2025-07-19 22:13:13 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

2025-07-19 22:13:13 INFO: Using device: cpu
2025-07-19 22:13:13 INFO: Loading: tokenize
2025-07-19 22:13:13 INFO: Loading: mwt
2025-07-19 22:13:13 INFO: Loading: pos
2025-07-19 22:13:15 INFO: Loading: lemma
2025-07-19 22:13:17 INFO: Done loading processors!


### **04. Carga y Exploración Inicial**


In [ ]:
df.head()

,region_nombre,provincia_nombre,comuna_nombre,cabildo_id,cabildo_nombre,cabildo_direccion,num_mesa,pregunta_glosa,concepto_nombre,concepto_otro,tipo-eleccion,fundamento,len_chars,len_tokens_raw
0,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Autonomía/libertad,NaN,A,sin fundamento,14,2
1,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Democracia,NaN,A,Sin fundamento,14,2
2,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Descentralización,NaN,A,La descentralizacion de ser autonoma por ser z...,66,11
3,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Equidad de género,NaN,A,Sin fundamento,14,2
4,Arica y Parinacota,Arica,Arica,81,Colegio San Marcos,"Santa María 2140, Arica",1,¿Cuáles son los valores y principios más impor...,Respeto/conservación de la naturaleza o medio ...,NaN,A,Sin fundamento,14,2


#### Conteo de nulos en columnas de texto


In [18]:
texto_col = "fundamento"
print(df[texto_col].isna().sum())

0


#### Estadísticas Descriptivas sobre el texto


In [23]:
df['len_chars'] = df[texto_col].str.len() # columna de longitud de caracteres
df['len_tokens_raw'] = df[texto_col].str.split().str.len() # columna de tokens sin procesar
df[['len_chars','len_tokens_raw']].describe()

,len_chars,len_tokens_raw
count,19326.000000,19326.000000
mean,100.842337,15.763014
std,68.723444,10.954642
min,1.000000,1.000000
25%,50.000000,8.000000
50%,84.000000,13.000000
75%,134.000000,21.000000
max,300.000000,61.000000


#### Eliminación de nulos y duplicados


In [24]:
before = len(df)
df = df.dropna(subset=[texto_col])
df = df.drop_duplicates(subset=[texto_col])
print(f"Filas eliminadas: {before - len(df)}")

Filas eliminadas: 0
